In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from Utils import Utils

csv_path = "/Users/shantanughosh/Desktop/Shantanu_MS/Research/Mattia_Prosperi/Propensity_Dropout/Dataset/ihdp_sample.csv"

from dataloader import DataLoader

dL = DataLoader()
split_size = 0.8
# np_covariates_X_train, np_covariates_X_test, \
# np_covariates_Y_train, np_covariates_Y_test = dL.preprocess_data_from_csv(csv_path, split_size=0.8)
# norm_X_train = np_covariates_X_train / np.linalg.norm(np_covariates_X_train)
# ps_train_set = dL.convert_to_tensor(np_covariates_X_train, np_covariates_Y_train)
# device = Utils.get_device()

# print(device)



In [2]:
np_covariates_X_train, np_covariates_X_test, np_covariates_Y_train, np_covariates_Y_test = \
                dL.preprocess_data_from_csv_augmented(csv_path, split_size)

.. Data Loading synthetic..
std dev
ps_np_covariates_X: (747, 77)
ps_np_treatment_Y: (747, 1)
np_covariates_X_train: (597, 77)
np_covariates_Y_train: (597, 1)
------------------------------------------------------------
np_covariates_X_test: (150, 77)
np_covariates_Y_test: (150, 1)


In [8]:
x = np.array([[1], [2], [3]])
y = np.array([[4, 5, 6, 9], [7, 8, 19, 18]])
y1 = y
np.random.seed(0)
std = np.std(y, axis=0)
print(std)



[1.5 1.5 6.5 4.5]


In [10]:
from datetime import datetime

# Current date time in local system
start = datetime.now()
print("start_time:" + str(start))

start_time:2020-06-28 02:31:07.035978


In [13]:
print("Duration : " + str( start - now))

Duration : 0:00:10.656428


In [9]:
now = datetime.now()
print("now:" + str(start))

now:2020-06-28 02:30:31.358836


In [11]:
np.random.seed(0)
print(std.shape[0])
y = np.array([[4, 5, 6, 9], [7, 8, 19, 18]])
id = -1
noise = np.empty([y.shape[0], y.shape[1]])
for s in std:
    id +=1
    print(s)
    np.random.seed(0)
#     noise[:, id] = s
    
    noise[:, id] = np.random.normal(0, 1.96 * s)
    
print(noise)

noise_y = noise + y

print(noise_y)

np.random.seed(0)
print(18 + np.random.normal(0, 1.96 * 4.5))    
    

4
1.5
1.5
6.5
4.5
[[ 5.1863139   5.1863139  22.47402689 15.55894169]
 [ 5.1863139   5.1863139  22.47402689 15.55894169]]
[[ 9.1863139  10.1863139  28.47402689 24.55894169]
 [12.1863139  13.1863139  41.47402689 33.55894169]]
33.558941691434796


In [12]:
np_covariates_X = np.concatenate((y, noise_y), axis=1)
np_covariates_X

array([[ 4.        ,  5.        ,  6.        ,  9.        ,  9.1863139 ,
        10.1863139 , 28.47402689, 24.55894169],
       [ 7.        ,  8.        , 19.        , 18.        , 12.1863139 ,
        13.1863139 , 41.47402689, 33.55894169]])

In [7]:
nrandom = np.random.random((747,25)) *10
nrandom

array([[6.46065812, 3.0567042 , 2.11719028, ..., 4.28878054, 3.10172903,
        1.54447345],
       [9.79916421, 6.72271051, 0.56458668, ..., 6.61807015, 0.84864102,
        2.20950882],
       [3.08468093, 0.80308587, 8.23004286, ..., 9.35602002, 4.00396147,
        3.45538455],
       ...,
       [7.6659666 , 5.92223383, 5.4396373 , ..., 2.00343179, 3.31540825,
        5.78755804],
       [7.03123107, 5.19232512, 0.0989919 , ..., 2.46743731, 8.43478922,
        1.385397  ],
       [6.85562166, 5.98248875, 0.6526608 , ..., 8.12321355, 2.73154856,
        0.28318412]])

In [ ]:

from Sparse_Propensity_score import Sparse_Propensity_score
train_parameters_SAE = {
            "epochs": 200,
            "lr": 0.0001,
            "batch_size": 32,
            "shuffle": True,
            "train_set": ps_train_set,
            "sparsity_probability": 0.08,
            "weight_decay": 0.0003,
            "BETA": 0.4,
            "model_save_path": "./Propensity_Model/SAE_PS_model_iter_id_"
                               + str(1) + "_epoch_{0}_lr_{1}.pth"
        }

# train_parameters_SAE = {
#     "epochs": 20,
#     "lr": 0.001,
#     "batch_size": 32,
#     "shuffle": True,
#     "train_set": ps_train_set,
#     "sparsity_probability": 0.05,
#     "weight_decay": 0.0003,
#     "BETA": 3,
#     "model_save_path": "./Propensity_Model/SAE_PS_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
# }

# train_parameters_SAE = {
#     "epochs": 200,
#     "lr": 0.0001,
#     "batch_size": 32,
#     "shuffle": True,
#     "train_set": ps_train_set,
#     "sparsity_probability": 0.08,
#     "weight_decay": 0.0003,
#     "BETA": 0.4,
#     "model_save_path": "./Propensity_Model/SAE_PS_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
# }

ps_net_SAE = Sparse_Propensity_score()
print("############### Propensity Score SAE net Training ###############")
sparse_classifier, sae_classifier_stacked_all_layer_active, sae_classifier_stacked_cur_layer_active = ps_net_SAE.train(train_parameters_SAE, device, phase="train")

data_loader = torch.utils.data.DataLoader(ps_train_set, shuffle=False, num_workers=4)



In [ ]:
sparse_classifier

In [ ]:
ps_score_list_SAE = ps_net_SAE.eval(ps_train_set, device, phase="eval", sparse_classifier=sparse_classifier)
ps_score_list_SAE_all_stacked = ps_net_SAE.eval(ps_train_set, device, phase="eval", 
                                                sparse_classifier=sae_classifier_stacked_all_layer_active)
ps_score_list_SAE_cur_stacked = ps_net_SAE.eval(ps_train_set, device, phase="eval", 
                                                sparse_classifier=sae_classifier_stacked_cur_layer_active)


In [ ]:
from Propensity_socre_network import Propensity_socre_network
train_parameters_NN = {
        "epochs": 100,
        "lr": 0.001,
        "batch_size": 32,
        "shuffle": True,
        "train_set": ps_train_set,
        "model_save_path": "./Propensity_Model/NN_PS_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
}
ps_net_NN = Propensity_socre_network()
print("############### Propensity Score neural net Training ###############")
ps_net_NN.train(train_parameters_NN, device, phase="train")

In [ ]:
# eval propensity network using NN
eval_parameters_NN = {
        "eval_set": ps_train_set,
        "model_path": "./Propensity_Model/NN_PS_model_iter_id_{0}_epoch_100_lr_0.001.pth".format(1)
}

ps_score_list_NN = ps_net_NN.eval(eval_parameters_NN, device, phase="eval")


In [ ]:
# ps_score_list_NN

In [ ]:
from DCN_network import DCN_network

def train_DCN(data_loader_dict, model_path, dL, device):
    treated_group = data_loader_dict["treated_data"]
    np_treated_df_X = treated_group[0]
    np_treated_ps_score = treated_group[1]
    np_treated_df_Y_f = treated_group[2]
    np_treated_df_Y_cf = treated_group[3]
    tensor_treated = dL.convert_to_tensor_DCN(np_treated_df_X, np_treated_ps_score,
                                              np_treated_df_Y_f, np_treated_df_Y_cf)

    control_group = data_loader_dict["control_data"]
    np_control_df_X = control_group[0]
    np_control_ps_score = control_group[1]
    np_control_df_Y_f = control_group[2]
    np_control_df_Y_cf = control_group[3]
    tensor_control = dL.convert_to_tensor_DCN(np_control_df_X, np_control_ps_score,
                                              np_control_df_Y_f, np_control_df_Y_cf)

    DCN_train_parameters = {
        "epochs": 100,
        "lr": 0.001,
        "treated_batch_size": 1,
        "control_batch_size": 1,
        "shuffle": True,
        "treated_set": tensor_treated,
        "control_set": tensor_control,
        "model_save_path": model_path
    }

    # train DCN network
    dcn = DCN_network()
    dcn.train(DCN_train_parameters, device)

In [ ]:
print("############### DCN Training using NN ###############")
data_loader_dict_NN = dL.prepare_tensor_for_DCN(np_covariates_X_train,
                                                    np_covariates_Y_train,
                                                    ps_score_list_NN)
model_path = "./DCNModel/NN_DCN_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
train_DCN(data_loader_dict_NN, model_path, dL, device)

In [ ]:
print("############### DCN Training using Sparse ###############")
data_loader_dict_SAE = dL.prepare_tensor_for_DCN(np_covariates_X_train,
                                                     np_covariates_Y_train,
                                                     ps_score_list_SAE)

data_loader_dict_SAE_all_stacked = dL.prepare_tensor_for_DCN(np_covariates_X_train,
                                                     np_covariates_Y_train,
                                                     ps_score_list_SAE_all_stacked)

data_loader_dict_SAE_cur_stacked = dL.prepare_tensor_for_DCN(np_covariates_X_train,
                                                     np_covariates_Y_train,
                                                     ps_score_list_SAE_cur_stacked)
print("e2e")
model_path = "./DCNModel/SAE_DCN_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
train_DCN(data_loader_dict_SAE, model_path, dL, device)





In [ ]:
print("all stacked")
model_path = "./DCNModel/SAE_all_stacked_DCN_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
train_DCN(data_loader_dict_SAE_all_stacked, model_path, dL, device)

In [ ]:
print("cur stacked")
model_path = "./DCNModel/SAE_cur_stacked_DCN_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
train_DCN(data_loader_dict_SAE_cur_stacked, model_path, dL, device)

In [ ]:
ps_test_set = dL.convert_to_tensor(np_covariates_X_test, np_covariates_Y_test)

In [ ]:
eval_parameters_sparse_test = {
        "eval_set": ps_test_set
}

ps_score_list_sparse = ps_net_SAE.eval(ps_test_set, device, phase="eval", sparse_classifier=sparse_classifier)
ps_score_list_sparse_all_stacked = ps_net_SAE.eval(ps_test_set, device, phase="eval", 
                                       sparse_classifier=sae_classifier_stacked_all_layer_active)
ps_score_list_sparse_cur_stacked = ps_net_SAE.eval(ps_test_set, device, phase="eval", 
                                       sparse_classifier=sae_classifier_stacked_cur_layer_active)


In [ ]:
# eval propensity network using NN
eval_parameters_NN = {
        "eval_set": ps_test_set,
        "model_path": "./Propensity_Model/NN_PS_model_iter_id_{0}_epoch_100_lr_0.001.pth".format(1)
}

ps_score_list_NN = ps_net_NN.eval(eval_parameters_NN, device, phase="eval")


In [ ]:

def do_test_DCN(data_loader_dict, dL, device, model_path):
    treated_group = data_loader_dict["treated_data"]
    np_treated_df_X = treated_group[0]
    np_treated_ps_score = treated_group[1]
    np_treated_df_Y_f = treated_group[2]
    np_treated_df_Y_cf = treated_group[3]
    tensor_treated = dL.convert_to_tensor_DCN(np_treated_df_X, np_treated_ps_score,
                                              np_treated_df_Y_f, np_treated_df_Y_cf)

    control_group = data_loader_dict["control_data"]
    np_control_df_X = control_group[0]
    np_control_ps_score = control_group[1]
    np_control_df_Y_f = control_group[2]
    np_control_df_Y_cf = control_group[3]
    tensor_control = dL.convert_to_tensor_DCN(np_control_df_X, np_control_ps_score,
                                              np_control_df_Y_f, np_control_df_Y_cf)

    DCN_test_parameters = {
        "treated_set": tensor_treated,
        "control_set": tensor_control,
        "model_save_path": model_path
    }

    dcn = DCN_network()
    err_dict = dcn.eval(DCN_test_parameters, device)
    err_treated = [ele ** 2 for ele in err_dict["treated_err"]]
    err_control = [ele ** 2 for ele in err_dict["control_err"]]

    total_sum = sum(err_treated) + sum(err_control)
    total_item = len(err_treated) + len(err_control)
    MSE = total_sum / total_item
    print("MSE: {0}".format(MSE))
    max_treated = max(err_treated)
    max_control = max(err_control)
    max_total = max(max_treated, max_control)

    min_treated = min(err_treated)
    min_control = min(err_control)
    min_total = min(min_treated, min_control)

    print("Max: {0}, Min: {1}".format(max_total, min_total))
    return MSE
    # np.save("treated_err.npy", err_treated)
    # np.save("control_err.npy", err_control)

In [ ]:
print("############### DCN Testing using NN ###############")
data_loader_dict_NN = dL.prepare_tensor_for_DCN(np_covariates_X_test,
                                                    np_covariates_Y_test,
                                                    ps_score_list_NN)
model_path = "./DCNModel/NN_DCN_model_iter_id_{0}_epoch_100_lr_0.001.pth".format(1)
MSE_NN = do_test_DCN(data_loader_dict_NN, dL, device, model_path)

In [ ]:
print("############### DCN Testing using SAE e2e ###############")
data_loader_dict_SAE = dL.prepare_tensor_for_DCN(np_covariates_X_test,
                                                     np_covariates_Y_test,
                                                     ps_score_list_sparse)
model_path = "./DCNModel/SAE_DCN_model_iter_id_{0}_epoch_100_lr_0.001.pth".format(1)
MSE_SAE = do_test_DCN(data_loader_dict_SAE, dL, device, model_path)


In [ ]:
print("############### DCN Testing using SAE all stacked ###############")
data_loader_dict_SAE = dL.prepare_tensor_for_DCN(np_covariates_X_test,
                                                     np_covariates_Y_test,
                                                     ps_score_list_sparse_all_stacked)
model_path = "./DCNModel/SAE_all_stacked_DCN_model_iter_id_{0}_epoch_100_lr_0.001.pth".format(1)
MSE_SAE = do_test_DCN(data_loader_dict_SAE, dL, device, model_path)


In [ ]:
print("############### DCN Testing using SAE cur stacked ###############")
data_loader_dict_SAE = dL.prepare_tensor_for_DCN(np_covariates_X_test,
                                                     np_covariates_Y_test,
                                                     ps_score_list_sparse_cur_stacked)
model_path = "./DCNModel/SAE_cur_stacked_DCN_model_iter_id_{0}_epoch_100_lr_0.001.pth".format(1)
MSE_SAE = do_test_DCN(data_loader_dict_SAE, dL, device, model_path)
